In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
pip install ninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 4.8 MB/s 


In [ ]:
pip install -U fastai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 4.7 MB/s 
     |████████████████████████████████| 209 kB 63.7 MB/s 
     |████████████████████████████████| 81 kB 9.2 MB/s 
     |████████████████████████████████| 78 kB 7.3 MB/s 
     |████████████████████████████████| 112 kB 66.3 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 147 kB 69.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=c2144fbc007330edfdd1ae2c0e39778c07f9f00c71ad08e492716baf56cb1193
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import os
os.environ['PYTHONHASHSEED']=str(1)
import random
from tensorflow import keras
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.layers import BatchNormalization,Dropout
from tensorflow.keras import models, layers, losses, optimizers, activations, regularizers
from tensorflow.keras.layers import LeakyReLU
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
import joblib
import pickle
from keras.engine.input_spec import InputSpec
initializer = tf.keras.initializers.Zeros()
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [ ]:
flow = np.load('/content/drive/MyDrive/CNN model data/lung_data/flow400.npy')
volume = np.load('/content/drive/MyDrive/CNN model data/lung_data/volume400.npy')
paw = np.load('/content/drive/MyDrive/CNN model data/lung_data/paw400.npy')
capacitances = np.load('/content/drive/MyDrive/CNN model data/lung_data/capacitances400.npy')
resistances = np.load("/content/drive/MyDrive/CNN model data/lung_data/rins400.npy")
#output1 = np.load("/content/drive/MyDrive/CNN model data/output1.npy")
#input1 = np.load("/content/drive/MyDrive/CNN model data/input1.npy")

In [ ]:
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(1)
   tf.random.set_seed(1)
   np.random.seed(1)
   random.seed(1)

In [ ]:
def normalize_data(data, minimum = None,maximum = None):
    if minimum is None:
        minimum = np.min(np.min(data))
    if maximum is None:
        maximum = np.max(np.max(data))
    data_norm = (data - minimum) / (maximum - minimum)
    return minimum, maximum, data_norm


In [ ]:
flow = flow.T
volume = volume.T
paw = paw.T
resistances = resistances.T
capacitances = capacitances.T

print("transposed")

num_examples = flow.shape[0]
num_samples = flow.shape[1]
print("num_samples are",num_examples)
(min_flow, max_flow, flow) = normalize_data(flow)
(min_volume, max_volume, volume) = normalize_data(volume)
(min_paw, max_paw, paw) = normalize_data(paw)
(min_resistance, max_resistance, resistances) = normalize_data(resistances)
(min_capacitance, max_capacitance, capacitances) = normalize_data(capacitances)

print("normalized data")

input_data = np.zeros((num_examples, num_samples, 3))
input_data[:, :, 0] = flow
input_data[:, :, 1] = volume
input_data[:, :, 2] = paw
output_data = np.concatenate((resistances, capacitances), axis=1)
indices = np.arange(num_examples)

print("input created")

input_train, input_test, output_train, output_test, indices_train, indices_test = \
    train_test_split(input_data, output_data, indices, test_size=0.3, shuffle=False, random_state=11)

input_validation, input_test, output_validation, output_test, indices_validation, indices_test = \
    train_test_split(input_test, output_test, indices_test, test_size=0.5, shuffle=False, random_state=11)

transposed
num_samples are 400
normalized data
input created


In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs
    print(res.shape)
    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(2)(x)
    return keras.Model(inputs, outputs)

In [ ]:
input_shape = input_train.shape[1:]

In [ ]:
def objective(trial):
    reset_random_seeds()
    params = {
              'learning_rate': trial.suggest_uniform('learning_rate', 1e-5, 1e-3),
              #'nb_filter': trial.suggest_categorical("nb_filter", [2, 3]),
              #'kernel_size': trial.suggest_int("kernel_size",2, 3, 5),
              'head_size': trial.suggest_int("head_size",128,256),
              'num_heads': trial.suggest_int("num_heads",3,4,5),
              'ff_dim': trial.suggest_int("ff_dim",2,3,4),
              'num_transformer_bock': trial.suggest_int("num_transformer_bock",3,4),
              'decay': trial.suggest_categorical("decay",[1e-3, 1e-5]),
              #'dense1': trial.suggest_int('dense1',50,100,150),
              #'dense2': trial.suggest_int('dense2',50,100,150),
              #'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              #'activation1': trial.suggest_categorical("activation1", ["relu", "LeakyReLU", "linear"]),
              #'activation2': trial.suggest_categorical("activation2", ["relu", "LeakyReLU", "linear"]),
              #'layers1': trial.suggest_categorical("layers1", [1,3]),
              #'layers2': trial.suggest_categorical("layers2", [1,3]),
              'dropout_rate': trial.suggest_categorical("dropout_rate", [0.0,0.3]),
              'epochs': trial.suggest_categorical("epochs", [100,150])
              }
    model = build_model(
    input_shape,
    head_size=params['head_size'],
    num_heads=params['num_heads'],
    ff_dim=params['ff_dim'],
    num_transformer_blocks=params['num_transformer_bock'],
    mlp_units=[128],
    mlp_dropout=0.0,
    dropout=0.0,
    )

    model.compile(optimizer=optimizers.Adam(lr=params['learning_rate'], decay=params['decay']), loss=losses.mean_squared_error)

    model.fit(input_train, output_train, epochs=params['epochs'], verbose=1,
            validation_data=(input_validation, output_validation)
            )

    y_pred = model.predict(input_test)
    mse = mean_squared_error(output_test, y_pred)
    mae = mean_absolute_error(output_test,y_pred)
    mape = mean_absolute_percentage_error(output_test,y_pred)

    with open("/content/drive/MyDrive/transformer_pk/{}.pickle".format(trial.number), "wb") as fout:
        pickle.dump(model, fout)
    return mse

In [ ]:
import optuna
reset_random_seeds()
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=21))
study.optimize(objective, n_trials=5)

[I 2022-11-12 16:19:43,428] A new study created in memory with name: no-name-acb2a962-e89b-4acf-abe8-699045e3fdf1
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:686: UserWarning: The distribution is specified by [3, 4] and step=5, but the range is not divisible by `step`. It will be replaced by [3, 3].
  low=low, old_high=old_high, high=high, step=step
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:686: UserWarning: The distribution is specified by [2, 3] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 2].
  low=low, old_high=old_high, high=high, step=step


Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


9/9 [==============================] - 39s 4s/step - loss: 0.7628 - val_loss: 0.3128
Epoch 2/100
9/9 [==============================] - 33s 4s/step - loss: 0.2547 - val_loss: 0.1670
Epoch 3/100
9/9 [==============================] - 33s 4s/step - loss: 0.1961 - val_loss: 0.1456
Epoch 4/100
9/9 [==============================] - 34s 4s/step - loss: 0.1552 - val_loss: 0.1026
Epoch 5/100
9/9 [==============================] - 34s 4s/step - loss: 0.1198 - val_loss: 0.0950
Epoch 6/100
9/9 [==============================] - 34s 4s/step - loss: 0.1085 - val_loss: 0.0858
Epoch 7/100
9/9 [==============================] - 34s 4s/step - loss: 0.0978 - val_loss: 0.0881
Epoch 8/100
9/9 [==============================] - 34s 4s/step - loss: 0.0901 - val_loss: 0.0784
Epoch 9/100
9/9 [==============================] - 34s 4s/step - loss: 0.0862 - val_loss: 0.0772
Epoch 10/100
9/9 [==============================] - 34s 4s/step - loss: 0.0828 - val_loss: 0.0751
Epoch 11/100
9/9 [=======================

[I 2022-11-12 17:14:34,491] Trial 0 finished with value: 0.06646088942762529 and parameters: {'learning_rate': 5.8237632001036017e-05, 'head_size': 165, 'num_heads': 3, 'ff_dim': 2, 'num_transformer_bock': 4, 'decay': 1e-05, 'dropout_rate': 0.3, 'epochs': 100}. Best is trial 0 with value: 0.06646088942762529.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:686: UserWarning: The distribution is specified by [3, 4] and step=5, but the range is not divisible by `step`. It will be replaced by [3, 3].
  low=low, old_high=old_high, high=high, step=step
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:686: UserWarning: The distribution is specified by

...

Epoch 1/150


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


9/9 [==============================] - 36s 4s/step - loss: 0.4508 - val_loss: 0.1514
Epoch 2/150
9/9 [==============================] - 31s 3s/step - loss: 0.1185 - val_loss: 0.0916
Epoch 3/150
9/9 [==============================] - 31s 3s/step - loss: 0.0851 - val_loss: 0.0805
Epoch 4/150
9/9 [==============================] - 31s 4s/step - loss: 0.0770 - val_loss: 0.0737
Epoch 5/150
9/9 [==============================] - 32s 4s/step - loss: 0.0671 - val_loss: 0.0677
Epoch 6/150
9/9 [==============================] - 31s 4s/step - loss: 0.0631 - val_loss: 0.0706
Epoch 7/150
9/9 [==============================] - 31s 4s/step - loss: 0.0611 - val_loss: 0.0665
Epoch 8/150
9/9 [==============================] - 32s 4s/step - loss: 0.0598 - val_loss: 0.0660
Epoch 9/150
9/9 [==============================] - 33s 4s/step - loss: 0.0577 - val_loss: 0.0725
Epoch 10/150
9/9 [==============================] - 32s 4s/step - loss: 0.0583 - val_loss: 0.0656
Epoch 11/150
9/9 [=======================

2/2 [==============================] - 3s 1s/step


[I 2022-11-12 21:09:20,543] Trial 4 finished with value: 0.07744068904884176 and parameters: {'learning_rate': 0.0008507997862573048, 'head_size': 162, 'num_heads': 3, 'ff_dim': 2, 'num_transformer_bock': 4, 'decay': 0.001, 'dropout_rate': 0.3, 'epochs': 150}. Best is trial 1 with value: 0.04537991875378236.


In [ ]:
trial = study.best_trial
print('MSE: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))
with open("/content/drive/MyDrive/transformer_pk/{}.pickle".format(study.best_trial.number), "rb") as fin:
    best_tcn = pickle.load(fin)
model = best_tcn
y_pred = model.predict(input_test)
mse = mean_squared_error(output_test, y_pred)
print(mse)
print(study.best_trial.number)

MSE: 0.04537991875378236
Best hyperparameters: {'learning_rate': 0.0005877553634564128, 'head_size': 136, 'num_heads': 3, 'ff_dim': 2, 'num_transformer_bock': 4, 'decay': 1e-05, 'dropout_rate': 0.3, 'epochs': 150}
2/2 [==============================] - 3s 1s/step
0.04537991875378236
1


In [ ]:
train_pred = model.predict(input_train)
train_nrmse = np.sqrt(np.sum((output_train - train_pred)**2))/np.sqrt(np.sum((output_train - np.average(output_train))**2))
print(train_nrmse)

In [ ]:
y_pred = model.predict(input_validation)
nrmse = np.sqrt(np.sum((output_validation - y_pred)**2))/np.sqrt(np.sum((output_validation - np.average(output_validation))**2))
print(nrmse)

2/2 [==============================] - 3s 1s/step
1.0030792546121376


In [ ]:
plt.scatter(output_validation[:, 0], y_pred[:, 0])